### Imports

In [ ]:
import sys
import os

project_path = os.path.abspath(os.path.join('../'))
if project_path not in sys.path:
    sys.path.append(project_path)

from utils.load_data import load_fer2013, parse_fer2013, \
                            load_CKPlus, fer_csv_to_png
from utils.preprocess import resnet_preprocess
from utils.generators import init_resnet_generator, init_cnn_generator, init_custom_generator
from utils.plots import plot_model_history
from models.build import build_resnet_model, build_cnn_baseline_model

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model

import numpy as np

### Loading datasets

In [ ]:
def unpack_csv(data_dir='../data/fer', file_path='../data/fer2013/fer2013.csv'):
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        os.mkdir(data_dir + '/train')
        os.mkdir(data_dir + '/val')
        os.mkdir(data_dir + '/test')
        fer_csv_to_png(file_path=file_path,
                       data_dir=data_dir)

# Local
unpack_csv(data_dir='../data/fer', file_path='../data/fer2013/fer2013.csv')

# Google colab
# unpack_csv(data_dir='/fer', file_path='/content/drive/MyDrive/fer2013.csv')

_, _, test_data = load_fer2013('../data/fer2013/fer2013.csv')
X_test, Y_test = parse_fer2013(test_data)
X_test = resnet_preprocess(X_test, target_size=(197, 197), target_channels=3)

### Model training parameters

In [ ]:
num_train = 28709
num_val = 3589
batch_size = 128
num_epochs = 150
patience = 20
saved_models_path = '../models/saved_models/'
dataset_name = 'fer2013'
model_name = '_resnet50_1'
history_name = '_hist' + model_name

# ../models/saved_models/fer2013_resnet50_1
model_path = saved_models_path + dataset_name + model_name

# ../models/saved_models/fer2013_hist_resnet50_1
history_path = saved_models_path + dataset_name + history_name

### Data generators

In [ ]:
train_gen = init_resnet_generator(data_dir='../data/fer/train', with_aug=True)
val_gen = init_resnet_generator(data_dir='../data/fer/val', with_aug=False)

### Callbacks

In [ ]:
cpt_path = model_path + '.hdf5'
early_stop = EarlyStopping('val_loss', patience=patience)
model_checkpoint = ModelCheckpoint(cpt_path, save_best_only=True)
callbacks = [model_checkpoint, early_stop]

### Build and compile model

In [ ]:
model = build_resnet_model()
opt = Adam(0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Training

In [ ]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=num_epochs,
                    steps_per_epoch=len(X_train)//batch_size,
                    validation_steps=len(X_val)//batch_size,
                    callbacks=callbacks)

# Saving final model and history
val_acc = history.history['val_accuracy'][-1]
cnt_epoch = len(history.history['val_accuracy'])
np.save(f'{history_path}-e{cnt_epoch:02d}-a{val_acc:.2f}.npy', history.history)
model.save(f'{model_path}-e{cnt_epoch:02d}-a{val_acc:.2f}.hdf5')

### Loading (if needed)

In [ ]:
# Uncomment to load
# history = np.load(history_path + '-e03-a0.41.npy', allow_pickle=True)
# history = history.item()
# model = load_model(model_path + '-e41-a0.71.hdf5')

### Evaluation

In [ ]:
loss, acc = model.evaluate(X_test, Y_test)
print("Model accuracy: {:5.2f}%".format(100 * acc))

### Ploting curves

In [ ]:
plot_model_history(history)